# Query data by geometry

In [ ]:
import sys
import json

# installing needed packages - this may take a few minutes
!conda install --yes --prefix {sys.prefix} shapely
!conda install --yes --prefix {sys.prefix} folium
import shapely.geometry
import folium

sys.path.append("..")
from pytreedb import db

Specify the file where database is stored locally additionally to MongoDB.

In [ ]:
mydbfile = "my_first_pytree.db"

Specify the location of the data to be imported. Here, we use a URL to a zipped folder with GeoJSON files for each tree to be added. You can also provide the path to a local directory containing the GeoJSON files.

In [ ]:
data_url = (
    "https://github.com/3dgeo-heidelberg/pytreedb/raw/main/data/test/geojsons.zip"
)

Define the (local) MongoDB connection and import data into pytreedb from URL.

In [ ]:
mydb = db.PyTreeDB(dbfile=mydbfile)
mydb.import_data(data_url, overwrite=True)

If we are only interested in trees in a certain area, we can query the tree database by a given geometry. For example, we can create a polygon and filter for trees lying within the polygon.

In [ ]:
SP02_coords = [
    [
        [8.6539, 49.0206],
        [8.6632, 49.0206],
        [8.6632, 49.0169],
        [8.6539, 49.0169],
        [8.6539, 49.0206],
    ]
]
poly_dict = json.dumps({"type": "Polygon", "coordinates": SP02_coords})

In [ ]:
geom = json.loads(poly_dict)
search_geom = shapely.geometry.shape(geom)

m = folium.Map([49.00772676282502, 8.701981844725482], zoom_start=12)
folium.GeoJson(search_geom).add_to(m)
m

In [ ]:
res = mydb.query_by_geometry(poly_dict)

In [ ]:
tree_locations = [tree["geometry"]["coordinates"] for tree in res]
tree_ids = [tree["properties"]["id"] for tree in res]

In [ ]:
m = folium.Map([49.0186, 8.6582], zoom_start=16)
folium.GeoJson(search_geom).add_to(m)
for point, name in zip(tree_locations, tree_ids):
    # note that we have to reverse latitude and longitude
    folium.Marker(point[1::-1], popup=name).add_to(m)
m

You can zoom in and click on the markers in the Map to read out the ID of the tree at the respective location.